# 動画とアノテーションデータからBBox付き動画を作成

In [1]:
import os
import json
import numpy as np
import cv2 as cv
import random
from timeit import default_timer as timer
from PIL import Image, ImageFont, ImageDraw

In [146]:
path_labels    = r'D:\SIGNATE\Signate_3rd_AI_edge_competition\train_annotations\train_00.json'   
path_images    = r'D:\SIGNATE\Signate_3rd_AI_edge_competition\images\train_00' 

classes = ['Car', 'Pedestrian']
#classes = ['車', '歩行者']

In [ ]:
import pandas as pd

#for i in range (0, len(annotations)):#here we browse all videos
#video_name = annotations[i].split('/')[-1].split('\\')[-1].split('.')[0]
data       = json.load(open(path_labels))
#print("\n",video_name)

#pd_data = pd.DataFrame(data)

ids_counter = np.zeros((4, 100000), dtype=int)

print("len(classes)=",len(classes))
Box_Frame = []
for v in range (0,600): #here we browse all frames. Single movie has 600 frames
    #img_name     = path_images+'/'+video_name+'/'+str(v)+".jpg"
    #img = cv.imread(img_name)
    BBoxes = []

    labels       = data['sequence'][v]
    #str_to_write = img_name
    for c in range (0, len(classes)):
        try:
            for inst in data['sequence'][v][classes[c]]:
                box           = inst['box2d']

                if ((box[2]-box[0])*(box[3]-box[1])) < 900: #we do not want small boxes
                    print("除外 :v={}, c={}, box[2]-box[0]={}, box[3]-box[1]={}".format(v,c,box[2]-box[0],box[3]-box[1]))
                    continue
                
                act_id        = inst['id']
                id_counter     = ids_counter[c,act_id]
                ids_counter[c,act_id] += 1

                BBox_Dict = {
                    "class": c, 
                    "id":act_id, 
                    "BBox":box
                    }
                
                BBoxes.append(BBox_Dict)

                #crop = normalize_image_size(img[box[1]:box[3], box[0]:box[2], ...], 224, 224)
                #print(" v={}, c={} :".format(v,c))
                #cv.imwrite(out_dir+'/'+str(c)+'_'+str(i)+'_'+str(act_id)+'_'+str(id_counter)+'_.jpg', crop)
                #out_file.write(out_dir+'/'+str(c)+'_'+str(i)+'_'+str(act_id)+'_'+str(id_counter)+'_.jpg\n')

        except Exception as e:
            print("v={}, c={} :{}".format(v,c,e))
            continue #nothing, the class is just not presented in the frame
            
    Box_Frame.append(BBoxes)    

In [158]:
import colorsys
hsv_tuples = [(x / len(classes), 1., 1.) for x in range(len(classes))]
colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
colors = list(map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)), colors))
colors = [(255, 0, 255), (0, 255, 255), (255, 255, 0), (0, 0, 255)]

In [159]:
print(colors)

[(255, 0, 255), (0, 255, 255), (255, 255, 0), (0, 0, 255)]


In [160]:
from PIL import Image, ImageFont, ImageDraw
def detect_image(image, BBox):
    start = timer()

    #print("image.size =",image.size[1])
    size1=np.floor(3e-2 * image.size[1] + 0.5).astype('int32')
    #print("size1 =",size1)

    font = ImageFont.truetype(font='fonts/arial.ttf', size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
    #font = ImageFont.truetype(size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
    thickness = (image.size[0] + image.size[1]) // 300


    for i in range(len(BBox)):
    #for i, c in reversed(list(enumerate(out_classes))):
        """
        predicted_class = self.class_names[i]
        box = out_boxes[i]
        score = out_scores[i]
        """
        class_no = BBox[i]["class"]
        predicted_class = classes[class_no]
        box = BBox[i]["BBox"]
        #score = out_scores[i]

        #label = '{}  id={}'.format(predicted_class, int(BBox[i]["id"]))
        label = '{}'.format(int(BBox[i]["id"]))
        draw = ImageDraw.Draw(image)
        label_size = draw.textsize(label, font)

        #top, left, bottom, right = box
        left, top, right, bottom = box
        #left, bottom, right, top = box
        top = max(0, np.floor(top + 0.5).astype('int32'))
        left = max(0, np.floor(left + 0.5).astype('int32'))
        bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
        right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
        print(label, (left, top), (right, bottom))

        if top - label_size[1] >= 0:
            text_origin = np.array([left, top - label_size[1]])
        else:
            text_origin = np.array([left, top + 1])

        # My kingdom for a good redistributable image drawing library.
        for i in range(thickness):
            draw.rectangle([left + i, top + i, right - i, bottom - i], outline=colors[class_no])

        draw.rectangle([tuple(text_origin), tuple(text_origin + label_size)], fill=colors[class_no])
        
        draw.text(text_origin, label, fill=(0, 0, 0), font=font)
        del draw

    end = timer()
    print(end - start)
    return image

In [161]:
def detect_video(video_path="D:\SIGNATE\Signate_3rd_AI_edge_competition\train_videos\train_00.mp4", output_path=r"D:\SIGNATE\Signate_3rd_AI_edge_competition\out_movie\base_00.mp4"):
    import cv2
    vid = cv2.VideoCapture(video_path)
    if not vid.isOpened():
        raise IOError("Couldn't open webcam or video")
    video_FourCC    = int(vid.get(cv2.CAP_PROP_FOURCC))
    video_fps       = vid.get(cv2.CAP_PROP_FPS)
    video_size      = (int(vid.get(cv2.CAP_PROP_FRAME_WIDTH)),
                        int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    print("video_fps={}, video_size={}".format(video_fps, video_size))
    isOutput = True if output_path != "" else False
    if isOutput:
        print("!!! TYPE:", type(output_path), type(video_FourCC), type(video_fps), type(video_size))
        out = cv2.VideoWriter(output_path, video_FourCC, video_fps, video_size)
    accum_time = 0
    curr_fps = 0
    fps = "FPS: ??"
    prev_time = timer()
    

    
    
    for v in range(0,600):
        return_value, frame = vid.read()
        img_name     = path_images + '/'+str(v)+".jpg"
        print("img_name = ",img_name)
        
        #image = cv2.imread(img_name)
        
        #data[v][classes[c]]
        image = Image.fromarray(frame)
        #image = Image.fromarray(image)
        image = detect_image(image, Box_Frame[v])
        
        result = np.asarray(image)
        
        curr_time = timer()
        exec_time = curr_time - prev_time
        prev_time = curr_time
        accum_time = accum_time + exec_time
        curr_fps = curr_fps + 1
        
        if accum_time > 1:
            accum_time = accum_time - 1
            fps = "FPS: " + str(curr_fps)
            curr_fps = 0
        cv2.putText(result, text="", org=(3, 15), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=0.50, color=(255, 0, 0), thickness=1)
        cv2.namedWindow("result", cv2.WINDOW_NORMAL)
        cv2.imshow("result", result)
        if isOutput:
            out.write(result)
            print("write(result)")
            
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("break")
            break
    

In [ ]:
detect_video(r"D:\SIGNATE\Signate_3rd_AI_edge_competition\train_videos\train_00.mp4")

In [112]:
from yolo import YOLO, detect_video